In [0]:
import subprocess
import sys

def install_package(package_name):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

install_package("faker")

from faker import Faker
fake = Faker()
print(fake.name())


  Obtaining dependency information for faker from https://files.pythonhosted.org/packages/08/9c/2bba87fbfa42503ddd9653e3546ffc4ed18b14ecab7a07ee86491b886486/Faker-33.1.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


Alyssa Williams


In [0]:
import os
import random
from random import choice
from datetime import datetime, timedelta
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("WebLogsProcessing").getOrCreate()

os.environ['TZ'] = 'Asia/Kolkata'
random.seed(42)

def generate_random_timestamp():
    start_time = datetime(2018, 1, 1, 0, 0, 0)
    end_time = datetime(2020, 1, 1, 0, 0, 0)
    delta = end_time - start_time
    random_seconds = random.randint(0, int(delta.total_seconds()))
    return start_time + timedelta(seconds=random_seconds)

num_records = 1_000_000

dictionary = {
    'request': ['GET', 'POST', 'PUT', 'DELETE'],
    'endpoint': ['/usr', '/usr/admin', '/usr/admin/developer', '/usr/login', '/usr/register'],
    'statuscode': ['303', '404', '500', '403', '502', '304', '200'],
    'statuscode-possible-pings': {
        '303': (0, 100), 
        '404': (100, 400), 
        '500': (400, 500), 
        '403': (500, 750), 
        '502': (750, 1000), 
        '304': (1001, 1240), 
        '200': (1240, 1500), 
    },
    'ua': [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:84.0) Gecko/20100101 Firefox/84.0',
        'Mozilla/5.0 (Android 10; Mobile; rv:84.0) Gecko/84.0 Firefox/84.0',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
        'Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Mobile Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4380.0 Safari/537.36 Edg/89.0.759.0',
        'Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.116 Mobile Safari/537.36 EdgA/45.12.4.5121',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36 OPR/73.0.3856.329',
        'Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Mobile Safari/537.36 OPR/61.2.3076.56749',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/7046A194A',
        'Mozilla/5.0 (iPhone; CPU iPhone OS 12_4_9 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Mobile/15E148 Safari/604.1'
    ],
    'referrer': ['-', 'http://example.com']
}

data = []
for i in range(num_records):
    status_code = choice(dictionary['statuscode'])
    
    (min_ping, max_ping) = dictionary['statuscode-possible-pings'][status_code]
    response_time = random.randint(min_ping, max_ping)

    data.append({
        'ip': f'192.168.1.{random.randint(1, 255)}',
        'timestamp': generate_random_timestamp(),
        'request_type': choice(dictionary['request']),
        'endpoint': choice(dictionary['endpoint']),
        'status_code': status_code,
        'response_time': response_time,
        'referrer': choice(dictionary['referrer']),
        'user_agent': choice(dictionary['ua']),
        'session_id': random.randint(1, 5000)
    })

df = pd.DataFrame(data)

spark_df = spark.createDataFrame(df)

parquet_output_dir = "/dbfs/tmp/requests-parts-parquet/"

spark_df.write.parquet(parquet_output_dir, mode='overwrite')

print(f"Data successfully written to Parquet at {parquet_output_dir}")

num_parts = 50
rows_per_part = len(df) // num_parts

for part_num in range(num_parts):
    start_row = part_num * rows_per_part
    end_row = start_row + rows_per_part if part_num < num_parts - 1 else len(df)
    part_df = df.iloc[start_row:end_row]

    part_spark_df = spark.createDataFrame(part_df)

    part_output_path = os.path.join(parquet_output_dir, f"part_{part_num + 1}")

    part_spark_df.write.parquet(part_output_path, mode='overwrite')

print(f"Data successfully split into {num_parts} parts and saved to Parquet at {parquet_output_dir}")


Data successfully written to Parquet at /dbfs/tmp/requests-parts-parquet/
Data successfully split into 50 parts and saved to Parquet at /dbfs/tmp/requests-parts-parquet/


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, window
from pyspark.sql.types import StructType, StringType, IntegerType, LongType, TimestampType

spark = SparkSession.builder \
    .appName("Web Logs Streaming Processing") \
    .getOrCreate()

schema = StructType() \
    .add("ip", StringType()) \
    .add("timestamp", TimestampType()) \
    .add("request_type", StringType()) \
    .add("endpoint", StringType()) \
    .add("status_code", StringType()) \
    .add("response_time", LongType()) \
    .add("referrer", StringType()) \
    .add("user_agent", StringType()) \
    .add("session_id", LongType())

streaming_df = spark.readStream \
    .option("maxFilesPerTrigger", 1) \
    .schema(schema) \
    .parquet(parquet_output_dir)

streaming_df.createOrReplaceTempView("original_stream")


In [0]:
%sql select * from original_stream

ip,timestamp,request_type,endpoint,status_code,response_time,referrer,user_agent,session_id
192.168.1.189,2018-01-05T16:04:57Z,DELETE,/usr/admin,200,1408,http://example.com,"Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Mobile Safari/537.36",2872
192.168.1.14,2018-04-05T16:41:55Z,GET,/usr/register,404,193,-,"Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.116 Mobile Safari/537.36 EdgA/45.12.4.5121",4984
192.168.1.247,2018-05-11T08:47:47Z,PUT,/usr/login,500,478,-,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36 OPR/73.0.3856.329",307
192.168.1.235,2019-09-07T21:32:44Z,DELETE,/usr/admin,200,1297,http://example.com,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:84.0) Gecko/20100101 Firefox/84.0,3935
192.168.1.182,2019-07-27T03:29:30Z,DELETE,/usr/login,303,25,-,"Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Mobile Safari/537.36 OPR/61.2.3076.56749",531
192.168.1.68,2018-10-29T22:42:43Z,DELETE,/usr/register,500,419,-,"Mozilla/5.0 (Linux; Android 10; ONEPLUS A6000) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.116 Mobile Safari/537.36 EdgA/45.12.4.5121",3120
192.168.1.207,2019-09-12T06:20:09Z,DELETE,/usr/admin,303,36,http://example.com,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36",2963
192.168.1.118,2018-02-11T12:21:49Z,DELETE,/usr/login,304,1051,http://example.com,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36",411
192.168.1.181,2018-07-02T09:43:31Z,POST,/usr,304,1084,-,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4380.0 Safari/537.36 Edg/89.0.759.0",4512
192.168.1.8,2019-07-06T10:24:05Z,GET,/usr/login,404,272,http://example.com,"Mozilla/5.0 (iPhone; CPU iPhone OS 12_4_9 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Mobile/15E148 Safari/604.1",1079


In [0]:
streaming_df = streaming_df.dropna(subset=["timestamp", "response_time"])

window_sizes = ["1 day", "7 days"]

for window_size in window_sizes:
    windowed_stats = streaming_df.groupBy(
        window(col("timestamp"), window_size),
        col("status_code")
    ).count()

    # windowed_stats.createOrReplaceTempView(f"window_stats_{window_size.replace(' ', '_')}")

    query = windowed_stats.writeStream \
        .outputMode("complete") \
        .format("memory") \
        .queryName(f"window_stats_{window_size.replace(' ', '_')}") \
        .start()

In [0]:
%sql select window.end as time, status_code, count from window_stats_1_day order by time

time,status_code,count
2018-01-02T00:00:00Z,303,8
2018-01-02T00:00:00Z,500,7
2018-01-02T00:00:00Z,403,5
2018-01-02T00:00:00Z,502,6
2018-01-02T00:00:00Z,404,6
2018-01-02T00:00:00Z,304,4
2018-01-02T00:00:00Z,200,10
2018-01-03T00:00:00Z,200,7
2018-01-03T00:00:00Z,303,6
2018-01-03T00:00:00Z,404,3


Databricks visualization. Run in Databricks to view.

In [0]:
%sql select window.end as time, status_code, count from window_stats_7_days order by time

time,status_code,count
2018-01-04T00:00:00Z,304,16
2018-01-04T00:00:00Z,200,21
2018-01-04T00:00:00Z,404,16
2018-01-04T00:00:00Z,502,19
2018-01-04T00:00:00Z,303,23
2018-01-04T00:00:00Z,403,16
2018-01-04T00:00:00Z,500,17
2018-01-11T00:00:00Z,403,44
2018-01-11T00:00:00Z,200,43
2018-01-11T00:00:00Z,303,39


Databricks visualization. Run in Databricks to view.

In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, MinMaxScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

indexer = StringIndexer(inputCol="status_code", outputCol="label")
assembler = VectorAssembler(inputCols=["response_time"], outputCol="features_raw")
scaler = MinMaxScaler(inputCol="features_raw", outputCol="features")
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)
pipeline = Pipeline(stages=[indexer, assembler, scaler, lr])

train_data, test_data = spark_df.randomSplit([0.8, 0.2], seed=1234)
model = pipeline.fit(train_data)
predictions = model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy: {accuracy:.2f}")

predictions.select("response_time", "status_code", "prediction").show()

Uploading artifacts:   0%|          | 0/4 [00:00<?, ?it/s]

Test Accuracy: 0.96
+-------------+-----------+----------+
|response_time|status_code|prediction|
+-------------+-----------+----------+
|         1442|        200|       5.0|
|          794|        502|       1.0|
|           35|        303|       0.0|
|          203|        404|       4.0|
|          126|        404|       4.0|
|         1049|        304|       6.0|
|          229|        404|       4.0|
|          128|        404|       4.0|
|         1046|        304|       6.0|
|         1164|        304|       6.0|
|         1437|        200|       5.0|
|          264|        404|       4.0|
|          253|        404|       4.0|
|          109|        404|       0.0|
|          726|        403|       2.0|
|         1491|        200|       5.0|
|          270|        404|       4.0|
|          728|        403|       2.0|
|           95|        303|       0.0|
|          488|        500|       3.0|
+-------------+-----------+----------+
only showing top 20 rows



In [0]:

status_code_to_label = model.stages[0].labels
predictions_stream = model.transform(streaming_df)

map_label_udf = udf(lambda label: status_code_to_label[int(label)], StringType())
predictions_stream = predictions_stream.withColumn("predicted_status_code", map_label_udf(col("prediction")))

predictions_stream = predictions_stream.groupBy("predicted_status_code", window(col("timestamp"), "1 day")).count()
query = predictions_stream.writeStream.outputMode("complete").format("memory").queryName("predictions").start()    

In [0]:
%sql select window.end as time, predicted_status_code, count from predictions order by time

time,predicted_status_code,count
2018-01-02T00:00:00Z,304,24
2018-01-02T00:00:00Z,502,28
2018-01-02T00:00:00Z,500,31
2018-01-02T00:00:00Z,200,33
2018-01-02T00:00:00Z,303,23
2018-01-02T00:00:00Z,403,28
2018-01-02T00:00:00Z,404,29
2018-01-03T00:00:00Z,403,22
2018-01-03T00:00:00Z,500,31
2018-01-03T00:00:00Z,502,25


Databricks visualization. Run in Databricks to view.